<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguage Natural

## Taller #10: Twitter
`Fecha de entrega: Noviembre 14, 2020 (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

###  `[25 pts]` Punto 1: Extraer Tuits
Extraer mínimo 100 tuits de Twitter usando la API

(Si exponen sus claves, les haré maldades 😈)

❌🎃❌ ¡No Halloween!

In [1]:
# importar librerias
import pandas as pd 
import tweepy

In [2]:
# Leer llaves
keys = pd.read_csv("D:/01. PC LENOVO/02. ESPECIALIZACION/Semestre 2/2. NLP/12 Clase/keys_demo.csv", header=None)
keys = dict(zip(keys[0],keys[1]))

In [3]:
# Autorizar a través API
auth = tweepy.OAuthHandler(keys['api_key'], keys['api_secret_key'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True)

In [4]:
#Recolectar twits de bogota
tweets = tweepy.Cursor(api.search,
                       q=['biden -RT'],
                       lang='es',
                       geocode=f'4.60971,-74.08175,45mi',
                       tweet_mode='extended',
                       include_rts=False).items(500)

In [5]:
t = []

for tweet in tweets:
    temp = []
    temp.append(tweet.full_text)
    temp.append(tweet.created_at)
    temp.append("Bogota")
    temp.append(tweet.user.location)
    temp.append(tweet.id)
    temp.append(f"https://twitter.com/i/web/status/{tweet.id}")
    
    t.append(temp)

In [6]:
data = pd.DataFrame(t)
data.columns = ['tweet', 'date', 'city', 'user_location', 'id', 'link']
data.head()

,tweet,date,city,user_location,id,link
0,"""Aún no lo han registrado"", pero seguramente s...",2020-11-11 02:21:53,Bogota,"Bogotá, D.C., Colombia",1326349396640993280,https://twitter.com/i/web/status/1326349396640...
1,"Si lo que pasa en Estados Unidos, con un Trump...",2020-11-11 02:19:43,Bogota,"Bogota, Colombia",1326348853868683264,https://twitter.com/i/web/status/1326348853868...
2,Caricatura que el papá de Biden le regaló cuan...,2020-11-11 02:16:36,Bogota,Bogotá,1326348069961609216,https://twitter.com/i/web/status/1326348069961...
3,Biden viendo a la canciller de Colombia defend...,2020-11-11 02:11:50,Bogota,"Bogotá, Colombia",1326346867077230599,https://twitter.com/i/web/status/1326346867077...
4,@Mara_mia2 @haidy_camargo @JMVivancoHRW Este i...,2020-11-11 02:09:16,Bogota,Bogota,1326346221477302272,https://twitter.com/i/web/status/1326346221477...


# Pre-procesamiento para Twitter

In [7]:
data = data[['tweet', 'date', 'city']]
data.head()

,tweet,date,city
0,"""Aún no lo han registrado"", pero seguramente s...",2020-11-11 02:21:53,Bogota
1,"Si lo que pasa en Estados Unidos, con un Trump...",2020-11-11 02:19:43,Bogota
2,Caricatura que el papá de Biden le regaló cuan...,2020-11-11 02:16:36,Bogota
3,Biden viendo a la canciller de Colombia defend...,2020-11-11 02:11:50,Bogota
4,@Mara_mia2 @haidy_camargo @JMVivancoHRW Este i...,2020-11-11 02:09:16,Bogota


In [8]:
# Quitar links, menciones, hashtags
# Quitar emojis
import re

pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
pattern2 = '?P<url>https?://[^\s]+'

def text_clean(row):
    text = row['tweet']
    
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]
    for link in links:
        text = text.replace(link,"")
             
    hashtags = [interaction for interaction in text.split() if interaction.startswith("#")]
    for hashtag in hashtags:
        text = text.replace(hashtag,"")
        
    mentions = [interaction for interaction in text.split() if interaction.startswith("@")]
    for mention in mentions:
        text = text.replace(mention,"")
        
    return text, links, hashtags, mentions

In [9]:
data[['clean', 'links', 'hashtags', 'mentions']] = data.apply(text_clean, axis=1, result_type='expand')

data.head()

,tweet,date,city,clean,links,hashtags,mentions
0,"""Aún no lo han registrado"", pero seguramente s...",2020-11-11 02:21:53,Bogota,"""Aún no lo han registrado"", pero seguramente s...",[https://t.co/pfbzNgRQV1],[],[]
1,"Si lo que pasa en Estados Unidos, con un Trump...",2020-11-11 02:19:43,Bogota,"Si lo que pasa en Estados Unidos, con un Trump...",[],[],[]
2,Caricatura que el papá de Biden le regaló cuan...,2020-11-11 02:16:36,Bogota,Caricatura que el papá de Biden le regaló cuan...,[https://t.co/Ev1r7gh1Fi],[],[]
3,Biden viendo a la canciller de Colombia defend...,2020-11-11 02:11:50,Bogota,Biden viendo a la canciller de Colombia defend...,[https://t.co/NtLRgLXVN7],[],[]
4,@Mara_mia2 @haidy_camargo @JMVivancoHRW Este i...,2020-11-11 02:09:16,Bogota,Este idiota se no sabrá que no hay adminis...,[],[],"[@Mara_mia2, @haidy_camargo, @JMVivancoHRW, @J..."


In [10]:
import regex
import emoji

def get_emojis(text):
    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)

    return emoji_list

data['emojis'] = data['clean'].apply(lambda text: get_emojis(text))

In [11]:
data.head()

,tweet,date,city,clean,links,hashtags,mentions,emojis
0,"""Aún no lo han registrado"", pero seguramente s...",2020-11-11 02:21:53,Bogota,"""Aún no lo han registrado"", pero seguramente s...",[https://t.co/pfbzNgRQV1],[],[],[]
1,"Si lo que pasa en Estados Unidos, con un Trump...",2020-11-11 02:19:43,Bogota,"Si lo que pasa en Estados Unidos, con un Trump...",[],[],[],[]
2,Caricatura que el papá de Biden le regaló cuan...,2020-11-11 02:16:36,Bogota,Caricatura que el papá de Biden le regaló cuan...,[https://t.co/Ev1r7gh1Fi],[],[],[]
3,Biden viendo a la canciller de Colombia defend...,2020-11-11 02:11:50,Bogota,Biden viendo a la canciller de Colombia defend...,[https://t.co/NtLRgLXVN7],[],[],[]
4,@Mara_mia2 @haidy_camargo @JMVivancoHRW Este i...,2020-11-11 02:09:16,Bogota,Este idiota se no sabrá que no hay adminis...,[],[],"[@Mara_mia2, @haidy_camargo, @JMVivancoHRW, @J...",[]


###  `[25 pts]` Punto 2: Hacer una visualización sobre esos tuits

❌🎃❌ ¡No Halloween!

In [12]:
data['fecha'] = data.date.dt.date
temp = pd.DataFrame(data.fecha.value_counts()).reset_index()
temp.columns = ['fecha', 'cnt']
temp = temp.sort_values('fecha')
temp

,fecha,cnt
0,2020-11-10,450
1,2020-11-11,50


In [13]:
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import matplotlib.pyplot as plt
%matplotlib inline
init_notebook_mode(connected=True)

In [14]:
trace = go.Scatter(x=temp.fecha.values,
                   y=temp.cnt.values,
                   text=[f"Fecha: {f}<br>Tuits: {c}" for f,c in zip(temp.fecha.values,temp.cnt.values)],
                   hoverinfo='text',
                   mode='lines+markers',
                   name='Días',
                   line={'color':'#1DA1F2'})


layout = go.Layout(title='Número de tuits por día')

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

In [15]:
from collections import Counter

temp_emojis = pd.DataFrame(list(zip(Counter(sum(data.emojis.values,[])).keys(),Counter(sum(data.emojis.values,[])).values())))
temp_emojis.columns = ['emoji', 'cnt']
temp_emojis.sort_values('cnt', ascending=False, inplace=True)
temp_emojis.head()

,emoji,cnt
25,🤣,8
11,👉,7
9,🇺🇸,6
5,🤔,5
8,🤭,5


In [16]:
trace = go.Bar(x=temp_emojis.emoji.values,
               y=temp_emojis.cnt.values)

layout = go.Layout(title='Emojis más usados')

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

In [17]:
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
stopwords_sp = stopwords.words('spanish')

def pre_procesado(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d_]+", " ", texto)
    texto = " ".join([palabra for palabra in texto.split() if palabra not in stopwords_sp])
    return texto

tfidf_vect = TfidfVectorizer(preprocessor=pre_procesado, ngram_range=(1,1), max_features=100)
tfidf = tfidf_vect.fit_transform(data.clean.values)

In [18]:
from sklearn.metrics.pairwise import cosine_similarity

voc = [k for k,v in sorted(tfidf_vect.vocabulary_.items(), key=lambda kv: kv[1])]
temp = pd.DataFrame(tfidf.toarray())
temp = cosine_similarity(temp.T.values)
temp = pd.DataFrame(temp)
temp.columns = voc
temp.index = voc
temp

,acuerdo,acuerdos,administración,agenda,ahora,aquí,arizona,así,años,aún,...,usted,ustedes,uu,va,van,veo,ver,vez,victoria,votos
acuerdo,1.0,0.000000,0.000000,0.000000,0.000000,0.068084,0.000000,0.000000,0.085446,0.035849,...,0.044376,0.000000,0.000000,0.042201,0.075374,0.000000,0.000000,0.000000,0.000000,0.000000
acuerdos,0.0,1.000000,0.076801,0.000000,0.000000,0.133568,0.000000,0.079550,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
administración,0.0,0.076801,1.000000,0.000000,0.000000,0.062605,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.128528,0.000000,0.000000,0.000000,0.038097,0.000000
agenda,0.0,0.000000,0.000000,1.000000,0.036869,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.069000,0.060824,0.000000,0.026737,0.073077,0.000000,0.000000,0.046568,0.000000,0.000000
ahora,0.0,0.000000,0.000000,0.036869,1.000000,0.000000,0.011263,0.055062,0.032712,0.000000,...,0.033719,0.000000,0.034779,0.135839,0.027283,0.000000,0.013083,0.000000,0.000000,0.026202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
veo,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.029454,0.000000,1.000000,0.000000,0.081836,0.000000,0.087385
ver,0.0,0.000000,0.000000,0.000000,0.013083,0.000000,0.019935,0.038733,0.000000,0.000000,...,0.085404,0.034268,0.000000,0.081801,0.000000,0.000000,1.000000,0.123909,0.076145,0.066878
vez,0.0,0.000000,0.000000,0.046568,0.000000,0.000000,0.000000,0.053190,0.000000,0.000000,...,0.000000,0.050016,0.000000,0.000000,0.028293,0.081836,0.123909,1.000000,0.000000,0.000000
victoria,0.0,0.000000,0.038097,0.000000,0.000000,0.000000,0.023401,0.000000,0.000000,0.035084,...,0.000000,0.000000,0.055372,0.000000,0.000000,0.000000,0.076145,0.000000,1.000000,0.000000


In [19]:
import scipy.cluster.hierarchy as sch
import numpy as np

pairwise_distances = sch.distance.pdist(temp)
linkage = sch.linkage(pairwise_distances, method='ward')
idx_to_cluster_array = sch.fcluster(linkage, pairwise_distances.max() * 1, criterion='distance')
idx = np.argsort(idx_to_cluster_array)
temp = temp.copy()
    
temp2 = temp.iloc[idx, :].T.iloc[idx, :]
my_idx = idx_to_cluster_array

In [20]:
trace = go.Heatmap(z=temp2.values.tolist(),
                   x=temp2.index.values,
                   y=temp2.columns.values,
                   colorscale='Oranges')

layout = go.Layout(title='🔥 Mapa de calor entre palabras (ordenado)🔥',
                   width=800, height=800)

fig = go.Figure(data=[trace],layout=layout)
iplot(fig)